# Tensorflowで文書を自動でタグ分類

doc2vec_test で作ったファイルを使い、新しい文書のタグを自動的に分類してみる

In [1]:
from ipywidgets.widgets import FloatProgress
from IPython.display import display

import time
import random
import numpy as np
from os import listdir, path
from gensim.models.doc2vec import Doc2Vec
import tensorflow as tf

BATCH_SIZE = 100
TRAIN_DATA_SIZE = 100
tf.reset_default_graph()

### 入力データを用意する

作成したトレーニングデータとテストデータをそれぞれ読み込み、バッチで取得するための準備を行う

`tf.train.shuffle_batch`はhttp://ykicisk.hatenablog.com/entry/2016/12/18/184840 が参考になった

shuffle_batchを使わずbatchでやった場合、同じカテゴリがまとまって入力されるため、偏った結果になってしまった。
shuffle_batchでランダムに取得することによって、確率的勾配降下法(SGD)になる

In [2]:
reader = tf.TFRecordReader()
min_after_dequeue = 5000  # 5000個以上キューが貯まるまで待ってそこからランダムに取得をするような感じだと思われる
capacity = min_after_dequeue + 3 * BATCH_SIZE


# トレーニングデータの準備
x_filename_queue = tf.train.string_input_producer(['../input/train.tfr'])
_, x_serialized_example = reader.read(x_filename_queue)
x_inputs = tf.parse_single_example(x_serialized_example, features={
    'id': tf.FixedLenFeature([1], tf.int64),
    'label': tf.FixedLenFeature([1], tf.int64),
    'feature': tf.FixedLenFeature([BATCH_SIZE], tf.float32),
})
x_batch = tf.train.shuffle_batch(x_inputs, batch_size=BATCH_SIZE, capacity=capacity, min_after_dequeue=min_after_dequeue)
print(x_batch)


# テストデータの準備
t_filename_queue = tf.train.string_input_producer(['../input/test.tfr'])
_, t_serialized_example = reader.read(t_filename_queue)
t_inputs = tf.parse_single_example(t_serialized_example, features={
    'id': tf.FixedLenFeature([1], tf.int64),
    'label': tf.FixedLenFeature([1], tf.int64),
    'feature': tf.FixedLenFeature([BATCH_SIZE], tf.float32),
})
t_batch = tf.train.shuffle_batch(t_inputs, batch_size=BATCH_SIZE, capacity=capacity, min_after_dequeue=min_after_dequeue)
print(t_batch)

{'feature': <tf.Tensor 'shuffle_batch:0' shape=(100, 100) dtype=float32>, 'id': <tf.Tensor 'shuffle_batch:1' shape=(100, 1) dtype=int64>, 'label': <tf.Tensor 'shuffle_batch:2' shape=(100, 1) dtype=int64>}
{'feature': <tf.Tensor 'shuffle_batch_1:0' shape=(100, 100) dtype=float32>, 'id': <tf.Tensor 'shuffle_batch_1:1' shape=(100, 1) dtype=int64>, 'label': <tf.Tensor 'shuffle_batch_1:2' shape=(100, 1) dtype=int64>}


### トレーニングデータを入れる変数
トレーニングデータの個数=None(無限)
トレーニングデータの要素数

In [3]:
x = tf.placeholder(tf.float32, [None, TRAIN_DATA_SIZE])
print(x.shape)

(?, 100)


### 重み
トレーニングデータの要素数
ラベルの種類数=9 (この場合のタグの種類は保存されているディレクトリの数)
0で初期化する。
重みは、トレーニングデータと学習アルゴリズムによって自動で獲得される

In [4]:
print(len([path.join('../data/text', x) for x in listdir('../data/text') if not x.endswith('.txt')]))

W = tf.Variable(tf.zeros([TRAIN_DATA_SIZE, 9]))

9


### バイアス
ラベル数の数だけ0で初期化
バイアスも、トレーニングデータと学習アルゴリズムによって自動で獲得される

In [5]:
b = tf.Variable(tf.zeros([9]))

### ソフトマックス関数
ソフトマックス関数の出力は0から1の間の実数になり、出力の総和は1となる
これにより文書が、各要素である確率を求めることができる

In [6]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

### 正解ラベルデータ
入力データとして取得した正解ラベルは0〜9の数字なので、9列のone hot配列に変換する
one hotは正解となる要素の列だけに値が入り他は０になるやつなので、
正解ラベルが`2`なら`[0, 5.0, 0, 0, 0, 0, 0, 0, 0]`みたいになる

In [7]:
one_hot = tf.placeholder(tf.int32, [None])
y_ = tf.one_hot(one_hot, depth=9, on_value = 5.0, off_value = 0.0, dtype=tf.float32)
print(y_.shape)

(?, 9)


### 交差エントロピー誤差
損失関数
正解ラベルとニューラルネットワークがどれだけ適合していないかを図るためのもの

In [8]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

### 勾配降下法を用い交差エントロピー誤差が最小となるようyを最適化する
0.01は学習率
学習率はハイパーパラメータといって、この値によってモデル全体の結果が変わってくる大事な値。
ハイパーパラメータは、重みやバイアスのように自動で獲得することはできない。

TODO: 色々変えて試してみる

In [9]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

### 学習させる
100個のバッチを1000回実行してみる

1エポック=`トレーニングデータをすべて使いきったときの回数`
今回の場合7,400件程度のデータの9割ほどをトレーニングデータとして使っているので、その件数分を回したら1エポックということらしい

HACK: 1エポックという単位が有るくらいだから、100エポックとか回したほうがいいのかな?
TODO: accuracyの変遷を見てもあまり学習できてない気がする?いろいろ試したい

In [10]:
fp = FloatProgress(min=0, max=1000)
display(fp)

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess = tf.Session()
sess.run(init_op)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
try:
    for i in range(1000):
        x_train = sess.run(x_batch)
        sess.run(train_step, feed_dict={x: x_train['feature'], one_hot: x_train['label'].reshape((BATCH_SIZE))})
        
        # 10件ごとに予測と精度の計算をする
        if i%10 == 0:
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            train_accuracy = sess.run(accuracy, feed_dict={x: x_train['feature'], one_hot: x_train['label'].reshape((BATCH_SIZE))})
            print("step %d, training accuracy %g"%(i, train_accuracy))
        fp.value = i
except tf.errors.OutOfRangeError:
    print('Done training for %d steps.' % (step))
finally:
    coord.request_stop()
    
coord.join(threads)

step 0, training accuracy 0.68
step 10, training accuracy 0.85
step 20, training accuracy 0.82
step 30, training accuracy 0.64
step 40, training accuracy 0.75
step 50, training accuracy 0.73
step 60, training accuracy 0.81
step 70, training accuracy 0.88
step 80, training accuracy 0.79
step 90, training accuracy 0.7
step 100, training accuracy 0.69
step 110, training accuracy 0.8
step 120, training accuracy 0.79
step 130, training accuracy 0.87
step 140, training accuracy 0.78
step 150, training accuracy 0.83
step 160, training accuracy 0.66
step 170, training accuracy 0.73
step 180, training accuracy 0.7
step 190, training accuracy 0.78
step 200, training accuracy 0.77
step 210, training accuracy 0.89
step 220, training accuracy 0.67
step 230, training accuracy 0.63
step 240, training accuracy 0.77
step 250, training accuracy 0.78
step 260, training accuracy 0.83
step 270, training accuracy 0.83
step 280, training accuracy 0.86
step 290, training accuracy 0.67
step 300, training accur

### テストデータを使って確認してみる

テストデータを500件取得して、それを使ってソフトマックス関数を実行してみる
重みとバイアスが更新されているはずなので、いい結果が出る?

TODO: 学習前に実行してみて、学習の効果を見てみたりしたい

In [11]:
t_train = sess.run(t_batch)

print("結果")

result = sess.run(y, feed_dict={x: t_train['feature']})

t_correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
t_accuracy = tf.reduce_mean(tf.cast(t_correct_prediction, "float"))
test_accuracy = sess.run(t_accuracy, feed_dict={x: t_train['feature'], one_hot: t_train['label'].reshape((BATCH_SIZE))})
print("test accuracy %g"%(test_accuracy))

print("\n1件だけ見てみる")

print("id      : {}".format(t_train['id'][0]))
print("label : {}".format(t_train['label'][0]))

print(result[0])
print("label : {}".format(np.argmax(result[0])))

結果
test accuracy 0.85

1件だけ見てみる
id      : [290]
label : [0]
[  9.82768357e-01   3.50690279e-05   1.37721945e-05   1.44424368e-04
   2.19040285e-05   2.19040285e-05   2.12749350e-04   5.95606631e-04
   1.61861740e-02]
label : 0


## 結果を見て

0.85という結果になりました。

低め8割くらいは正しい分類ができるってことでしょうか。
高く感じますが、5回1回失敗する機能と考えるとあまり使い物にならないですね。